In [2]:
import os
# 【硬性要求】注入环境变量 (确保读取到 OPENAI_API_KEY)
from dotenv import load_dotenv
load_dotenv(override=True)

# 引入核心模块
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# ==========================================
# 1. 索引阶段 (Indexing Phase)
# ==========================================
# 指向你的目标 PDF 文件路径
PDF_PATH = "data/handbook.pdf"

print(f"1. 正在加载 PDF 文件: {PDF_PATH} ...")
try:
    # 实例化加载器并按页粗略分割
    loader = PyPDFLoader(PDF_PATH)
    pages = loader.load_and_split()
    
    # 细粒度文本分块 (Chunking)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=50)
    chunks = text_splitter.split_documents(pages)
    
    print("2. 正在进行向量化并存入 Chroma 数据库...")
    # 初始化 Embeddings 模型并建立向量数据库
    embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
    vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings)
    
    # ==========================================
    # 2. 检索与生成组件设置 (Retrieval & Generation Setup)
    # ==========================================
    # 将向量库转化为检索器 (Retriever)
    retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
    
    # 定义辅助函数：拼接检索到的文档块为纯文本字符串
    def format_docs(retrieved_docs):
        return "\n\n".join(doc.page_content for doc in retrieved_docs)
        
    # 初始化大语言模型 (LLM)
    llm = ChatOpenAI(model="gpt-4o", temperature=0)
    
    # 定义防幻觉提示词模板 (Prompt Template)
    template = """SYSTEM: You are a question answer bot.
    Be factual in your response.
    Respond to the following question ONLY from the below context: 
    {context}
    
    If you don't know the answer, just say that you don't know.
    
    Question: {question}
    """
    prompt = PromptTemplate.from_template(template)
    
    # ==========================================
    # 3. 组装 LCEL 终极管道 (Assembling the Chain)
    # ==========================================
    print("3. 正在组装 LCEL 端到端问答管道...")
    rag_chain = (
        # 魔法时刻：并行处理 context 和 question
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )
    
    # ==========================================
    # 4. 执行测试 (Execution)
    # ==========================================
    print("\n=== RAG 系统已上线 ===")
    
    # 测试老师讲稿中的经典问题
    test_question = "According to the author, in the contemporary digital landscape, the \"datasphere\" is expanding at what rate?"
    print(f"\n🙋 提问: {test_question}")
    
    response = rag_chain.invoke(test_question)
    print(f"🤖 回答: {response}")

except FileNotFoundError:
    print(f"\n❌ 错误：找不到文件 '{PDF_PATH}'。请确保在当前目录下创建了 'data' 文件夹，并放入了名为 'handbook.pdf' 的文件。")
except Exception as e:
    print(f"\n❌ 运行发生异常: {e}")

1. 正在加载 PDF 文件: data/handbook.pdf ...
2. 正在进行向量化并存入 Chroma 数据库...
3. 正在组装 LCEL 端到端问答管道...

=== RAG 系统已上线 ===

🙋 提问: According to the author, in the contemporary digital landscape, the "datasphere" is expanding at what rate?
🤖 回答: The "datasphere" is expanding at an unprecedented rate.
